In [1]:
# !pip install sqlalchemy

In [1]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain

In [2]:


# Load environment variables from .env file
load_dotenv()

# Fetch API key from environment variable
google_api = os.getenv("google")
# Initialize the LLM
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=google_api)
# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=palm)

db_path = "C:/Users/Omkar/OneDrive/Desktop/SQL_chat_assistant/demodb.db"
db_uri = f"sqlite:///{db_path}"

print(db_uri)

sqlite:///C:/Users/Omkar/OneDrive/Desktop/SQL_chat_assistant/demodb.db


In [3]:
# Create SQLDatabase instance with sample rows
db = SQLDatabase.from_uri(
    db_uri,
    include_tables=['Employees', 'Departments'],  # Include your table names
    sample_rows_in_table_info=3
)

# Print table info (will show schema + sample rows similar to your example)
print(db.table_info)


CREATE TABLE "Departments" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Manager" TEXT NOT NULL, 
	PRIMARY KEY ("ID")
)

/*
3 rows from Departments table:
ID	Name	Manager
1	Sales	Alice
2	Engineering	Bob
3	Marketing	Charlie
*/


CREATE TABLE "Employees" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Department" TEXT NOT NULL, 
	"Salary" INTEGER NOT NULL, 
	"Hire_Date" TEXT NOT NULL, 
	PRIMARY KEY ("ID"), 
	FOREIGN KEY("Department") REFERENCES "Departments" ("Name")
)

/*
3 rows from Employees table:
ID	Name	Department	Salary	Hire_Date
1	Alice	Sales	50000	2021-01-15
2	Bob	Engineering	70000	2020-06-10
3	Charlie	Marketing	60000	2022-03-20
*/


In [12]:
# # Create SQL query chain
# chain = create_sql_query_chain(llm, db)

# # Example usage
# question = "Show me all employees in the IT department"
# query = chain.invoke({"question": question})

# print("\nGenerated SQL Query:")
# print(query)

# cleaned_query = query.strip('```sql\n').strip('\n```')
# print("\nGenerated cleaned SQL Query:")
# print(cleaned_query)


Generated SQL Query:
```sql
SELECT 
    "Name"
FROM 
    "Employees"
WHERE 
    "Department" = 'IT';
```

Generated cleaned SQL Query:
SELECT 
    "Name"
FROM 
    "Employees"
WHERE 
    "Department" = 'IT';


In [13]:
# # Execute the query using the SQLDatabase instance
# result = db.run(cleaned_query)
# print("\nQuery Results:")
# print(result)


Query Results:
[('Frank',)]


In [13]:
# Create SQL query chain
chain = create_sql_query_chain(llm, db)

def execute_query(question):
    try:
        # Generate SQL query
        query = chain.invoke({"question": question})
        print("\nGenerated SQL Query:")
        print(query)

        # Clean the query (remove ```sql and extra spaces)
        cleaned_query = query.strip('```sql\n').strip('\n```')
        print("\nCleaned SQL Query:")
        print(cleaned_query)

        # Execute the query
        result = db.run(cleaned_query)
        
        # Handle empty results
        if not result:
            return "No results found for your query."

        finalresult=llm("frame simple short sentance with following answer:-" + result + " with refernce to thi following question:-" + question)
        return finalresult

    except Exception as e:
        # Handle invalid queries or execution errors
        error_message = str(e)
        if "no such table" in error_message.lower():
            return "Error: The specified table does not exist."
        elif "no such column" in error_message.lower():
            return "Error: The specified column does not exist."
        elif "syntax error" in error_message.lower():
            return "Error: The generated SQL query is invalid."
        else:
            return f"An error occurred: {error_message}"


In [16]:

# Example usage
question = "Who is the manager of sales department"
result = execute_query(question)
print("\nQuery Results:")
print(result)


Generated SQL Query:
<think>
Okay, so I need to figure out who the manager of the Sales department is. Let me start by looking at the tables provided. There's a "Departments" table and an "Employees" table. 

First, I'll check the "Departments" table. It has columns ID, Name, and Manager. Since the question is about the Sales department, I should look for the row where the Name is 'Sales'. In that row, the Manager column should give me the answer.

Looking at the data, the Departments table has three rows. The first row has ID 1, Name 'Sales', and Manager 'Alice'. So, that's straightforward. The manager is Alice.

Wait, but I also see an Employees table. Let me check if Alice is an employee there. Yes, the first row in Employees has ID 1, Name 'Alice', Department 'Sales', Salary 50000, and Hire_Date '2021-01-15'. So, Alice is indeed an employee in the Sales department, which aligns with her being the manager as per the Departments table.

I don't think I need to join the tables or any

### different langchain

In [6]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db)
question = "Show me all employees in the IT department"
qns1 = db_chain( question + "and dont give query in sql bash just simple query")
print(qns1)

C:\Users\Omkar\AppData\Local\Temp\ipykernel_17940\2749753123.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qns1 = db_chain( question + "and dont give query in sql bash just simple query")


OperationalError: (sqlite3.OperationalError) near "```sql
SELECT "Name"
FROM "Employees"
WHERE "Department" = 'IT'
LIMIT 5;
```": syntax error
[SQL: ```sql
SELECT "Name"
FROM "Employees"
WHERE "Department" = 'IT'
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)